In [3]:
import pandas as pd

all_df = pd.read_csv(r'test_sale.csv')
all_df

,Unnamed: 0,DateTime,Final price,Sale Start,Sale End,Origin Price,Sale Rate,Ready to Sale,Sale Period,Game_ID,Start Year,Start month,Start day,End Year,End month,End day
0,0,2015-11-25 18:24:05,5250.0,2015-11-25 18:24:05,2015-12-01 18:39:34,10500.0,0.50,21 days 00:54:41,6 days 00:15:29,10,2015,11,25,2015,12,1
1,1,2015-12-22 19:34:15,5250.0,2015-12-22 19:34:15,2016-01-04 19:21:20,10500.0,0.50,32 days 00:10:02,12 days 23:47:05,10,2015,12,22,2016,1,4
2,2,2016-02-05 19:31:22,2620.0,2016-02-05 19:31:22,2016-02-12 18:47:14,10500.0,0.75,285 days 01:47:07,6 days 23:15:52,10,2016,2,5,2016,2,12
3,3,2016-11-23 20:34:21,2620.0,2016-11-23 20:34:21,2016-12-01 16:14:00,10500.0,0.75,21 days 01:44:51,7 days 19:39:39,10,2016,11,23,2016,12,1
4,4,2016-12-22 17:58:51,2620.0,2016-12-22 17:58:51,2017-01-02 18:23:32,10500.0,0.75,171 days 02:48:31,11 days 00:24:41,10,2016,12,22,2017,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,22,2023-06-29 17:29:52,11520.0,2023-06-29 17:29:52,2023-07-13 17:24:00,34900.0,0.67,24 days 23:58:44,13 days 23:54:08,999020,2023,6,29,2023,7,13
26044,23,2023-08-07 17:22:44,11520.0,2023-08-07 17:22:44,2023-08-21 17:22:32,34900.0,0.67,29 days 23:39:43,13 days 23:59:48,999020,2023,8,7,2023,8,21
26045,24,2023-09-20 17:02:15,11520.0,2023-09-20 17:02:15,2023-10-04 17:02:11,34900.0,0.67,34 days 00:59:45,13 days 23:59:56,999020,2023,9,20,2023,10,4
26046,25,2023-11-07 18:01:56,11520.0,2023-11-07 18:01:56,2023-11-28 18:29:29,34900.0,0.67,22 days 23:40:53,21 days 00:27:33,999020,2023,11,7,2023,11,28


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

all_df = pd.read_csv(r'test_sale.csv')
game_info_df = pd.read_csv(r'game_account_merge.csv')


# 게임 정보에서 값 가져오기
game_info_df = game_info_df.drop(columns=['DLC_Game'])
#pd.to_datetime(datetime.today()) - pd.to_datetime(game_info_df['release date'])
#test_day = game_info_df['release date'].iloc[0]

# 게임정보 필요 columns 정규화
game_info_df['release date'] = game_info_df['release date'].str.replace(',','')
temp_index = game_info_df[game_info_df['release date'].apply(lambda x : len(x.split())) < 3].index
game_info_df['release date'].iloc[temp_index] = '1 ' + game_info_df['release date'].iloc[temp_index]

game_info_df['last Day'] = (pd.to_datetime(datetime.today()) - pd.to_datetime(game_info_df['release date'])).apply(lambda x : str(x).split()[0])
game_info_df = game_info_df.drop(columns=['Unnamed: 0','Name','release date'])
game_info_df['Company'] = game_info_df['Company'].apply(lambda x : x.split(',')[0])

# 파일 불러오기
df = pd.read_csv(r'test_sale.csv')
df['Ready to Sale Day'] = df['Ready to Sale'].apply(lambda x : x.split()[0])
df['Sale Peroid Day'] = df['Sale Period'].apply(lambda x : int(x.split()[0])+1)

# 필요없는 값 드랍
temp_df = df.drop(columns=['Ready to Sale','Origin Price','Sale Rate','Sale Period','Sale End','DateTime'])

# game정보와 변환 데이터 합치기
test_df = pd.merge(temp_df, game_info_df,left_on='Game_ID', right_on='AppID', how='right')

# Nan 값처리
test_df['price'] = test_df['price'].str.extract('(\d+)')
test_df['price'] = test_df['price'].fillna(0)

test_df['Ready to Sale Day'] = test_df['Ready to Sale Day'].fillna(0)
test_df['Sale Peroid Day'] = test_df['Sale Peroid Day'].fillna(0)

test_df = test_df.dropna()

# 타입 형 형변환

test_df['Unnamed: 0'] = test_df['Unnamed: 0'].astype(int) + 1


test_df['Start Year'] = test_df['Start Year'].astype('int16')
test_df['Start month'] = test_df['Start month'].astype('int16')
test_df['Start day'] = test_df['Start day'].astype('int16')

test_df['End Year'] = test_df['End Year'].astype('int16')
test_df['End month'] = test_df['End month'].astype('int16')
test_df['End day'] = test_df['End day'].astype('int16')

test_df['Start Year'] = test_df['Start Year'].astype(object)
test_df['Start month'] = test_df['Start month'].astype(object)
test_df['Start day'] = test_df['Start day'].astype(object)

test_df['End Year'] = test_df['End Year'].astype(object)
test_df['End month'] = test_df['End month'].astype(object)
test_df['End day'] = test_df['End day'].astype(object)
test_df['Sale Start'] = pd.to_datetime(test_df['Sale Start'])

test_df['price'] = test_df['price'].astype('int32')
test_df['last Day'] = test_df['last Day'].astype('int32')

test_df['Game_ID'] = test_df['Game_ID'].astype(int)
test_df['Final price'] = test_df['Final price'].astype('int32')
test_df['Sale Peroid Day'] = test_df['Sale Peroid Day'].astype('int32')
test_df['Ready to Sale Day'] = test_df['Ready to Sale Day'].astype('int32')

test_df['latest rating counter'] = test_df['latest rating counter'].astype('int32')
test_df['all rating counter'] = test_df['all rating counter'].astype('int32')

# 마지막 데이터 변환
test_df = test_df.drop(['AppID'],axis=1)
test_df = test_df.reset_index()
test_df = test_df.drop(['index'],axis=1)

test_df = test_df.drop(['End day','Start day','Company','category','Developer'],axis=1)
test_df

<>:35: SyntaxWarning: invalid escape sequence '\d'
<>:35: SyntaxWarning: invalid escape sequence '\d'
C:\Users\GJ\AppData\Local\Temp\ipykernel_4052\1980826346.py:35: SyntaxWarning: invalid escape sequence '\d'
  test_df['price'] = test_df['price'].str.extract('(\d+)')
C:\Users\GJ\AppData\Local\Temp\ipykernel_4052\1980826346.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_info_df['release date'].iloc[temp_index] = '1 ' + game_info_df['release date'].iloc[temp_index]


,Unnamed: 0,Final price,Sale Start,Game_ID,Start Year,Start month,End Year,End month,Ready to Sale Day,Sale Peroid Day,price,latest rating,latest rating counter,all rating,all rating counter,Total_Gross,Total_Net,Total_owners,last Day
0,1,5250,2015-11-25 18:24:05,10,2015,11,2015,12,21,7,10500,Overwhelmingly Positive,1267,Overwhelmingly Positive,146475,50000000.0,30000000.0,9300000.0,8496
1,2,5250,2015-12-22 19:34:15,10,2015,12,2016,1,32,13,10500,Overwhelmingly Positive,1267,Overwhelmingly Positive,146475,50000000.0,30000000.0,9300000.0,8496
2,3,2620,2016-02-05 19:31:22,10,2016,2,2016,2,285,7,10500,Overwhelmingly Positive,1267,Overwhelmingly Positive,146475,50000000.0,30000000.0,9300000.0,8496
3,4,2620,2016-11-23 20:34:21,10,2016,11,2016,12,21,8,10500,Overwhelmingly Positive,1267,Overwhelmingly Positive,146475,50000000.0,30000000.0,9300000.0,8496
4,5,2620,2016-12-22 17:58:51,10,2016,12,2017,1,171,12,10500,Overwhelmingly Positive,1267,Overwhelmingly Positive,146475,50000000.0,30000000.0,9300000.0,8496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26010,23,13000,2023-03-16 17:29:10,985890,2023,3,2023,3,98,7,26000,Very Positive,82,Very Positive,14585,6000000.0,4000000.0,300000.0,1376
26011,24,13000,2023-06-29 17:29:53,985890,2023,6,2023,7,55,14,26000,Very Positive,82,Very Positive,14585,6000000.0,4000000.0,300000.0,1376
26012,25,13000,2023-09-07 10:00:04,985890,2023,9,2023,9,64,11,26000,Very Positive,82,Very Positive,14585,6000000.0,4000000.0,300000.0,1376
26013,26,13000,2023-11-21 18:12:19,985890,2023,11,2023,11,22,8,26000,Very Positive,82,Very Positive,14585,6000000.0,4000000.0,300000.0,1376


In [ ]:
test_df.info()

---
# 기본 데이터 8:2로 선정

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import pickle

# 기본 테스트 데이터 최종만 선정
test_temp_df = test_df.drop(columns=['Sale Start'],axis=1)
running_df = pd.get_dummies(test_temp_df)
running_df= running_df.dropna()

x = running_df.drop(['Final price'],axis=1)
y = running_df['Final price']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state=111)

# sklearn 에 적재되어 있는 알고리즘
#from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import GaussianNB

#from sklearn.ensemble import VotingClassifier
#from sklearn.ensemble import BaggingClassifier
#from sklearn.ensemble import RandomForestClassifier

# 설치가 필요한 앙상블 알고리즘 
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.ensemble import GradientBoostingClassifier
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier

# 평가함수
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# 분류 기본 알고리즘
#tree = DecisionTreeClassifier()     # 의사결정트리
logistic = LogisticRegression()     # 로지스틱회귀
#svc = SVC(probability=True)         # SVC
knn = KNeighborsClassifier()        # KMean
#nb = GaussianNB()
#cat = CatBoostClassifier()

# Ensemble
#rfc = RandomForestClassifier(n_estimators=1000)         # 랜덤 포래스트
#bag = BaggingClassifier(n_estimators=100)               # bagging
#ada = AdaBoostClassifier(n_estimators=500)              # AdaBoost
#grad = GradientBoostingClassifier(n_estimators=500)     # GradientBoost
#xgb = XGBClassifier(n_estimators=500)                   # xgbBoost
#lgbm = LGBMClassifier()                                 # lightgbBoost

#models = [ tree,logistic,svc, knn, nb, rfc, bag, ada, grad, xgb, lgbm ]
models = [logistic,knn]

#models = [rfc,xgb,lgbm]
check_df = pd.DataFrame(columns=['model','score','time'])

data_model = dict()
data_confirm = dict()
data_preds = dict()
from sklearn.metrics import classification_report


model_path = 'ml_model'

# 1차 검색 테스트
#for model in models:
#    print('{} model 시작'.format(model))
#
#    #try:
#    timer = time.time()
#    model.fit(x_train,y_train)
#    preds = model.predict(x_test)
#    score = accuracy_score(y_test, preds)
#    timer = time.time() - timer
#    check_df.loc[len(check_df)] = [model.__class__.__name__, score, round(timer,3)]
#    data_model[model.__class__.__name__] = model
#    data_confirm[model.__class__.__name__] = classification_report(y_test, preds)
#    data_preds[model.__class__.__name__] = preds
#    with open(os.path.join(model_path,'{}.pickle'.format(model.__class__.__name__)),'wb') as f:
#        pickle.dump(model, f)
#    #except:
#    #    print('error model : {}'.format(model.__class__.__name__))




check_df

NameError: name 'test_df' is not defined

---
# 마지막 데이터만 TEST 로 선정

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import pickle

# 1안 마지막 데이터만 선정
last_count_index = test_df.reset_index().sort_values('Sale Start').groupby('Game_ID')[['price','index']].last()['index'].unique()
running_df = pd.get_dummies(test_df)

train_running_df = running_df.iloc[last_count_index]
test_running_df = running_df.drop(last_count_index)

train_running_df = train_running_df.dropna()
test_running_df = test_running_df.dropna()
#running_df.iloc[last_count_index]   # 마지막 데이터
#running_df.drop(last_count_index)   # 그 외 데이터

x_train = train_running_df.drop(['Final price','Sale Start'],axis=1)
y_train = train_running_df['Final price']

x_test = test_running_df.drop(['Final price','Sale Start'],axis=1)
y_test = test_running_df['Final price']


# sklearn 에 적재되어 있는 알고리즘
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

# 설치가 필요한 앙상블 알고리즘 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# 평가함수
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# 분류 기본 알고리즘
tree = DecisionTreeClassifier()     # 의사결정트리
logistic = LogisticRegression()     # 로지스틱회귀
svc = SVC(probability=True)         # SVC
knn = KNeighborsClassifier()        # KMean
nb = GaussianNB()
#cat = CatBoostClassifier()

# Ensemble
rfc = RandomForestClassifier(n_estimators=1000)         # 랜덤 포래스트     0.8
bag = BaggingClassifier(n_estimators=100)               # bagging           # 0.2
ada = AdaBoostClassifier(n_estimators=500)              # AdaBoost          # 0.2
grad = GradientBoostingClassifier(n_estimators=500)     # GradientBoost     0.8
xgb = XGBClassifier(n_estimators=500)                   # xgbBoost          # 인식 X
lgbm = LGBMClassifier()                                 # lightgbBoost      # 인식 X

#models = [ tree,logistic,svc, knn, nb, rfc, bag, ada, grad, xgb, lgbm ]
models = [logistic,knn]

#models = [rfc,xgb,lgbm]
check_df = pd.DataFrame(columns=['model','score','time'])

#check_df.loc[len(check_df)] = [model.__class__.__name__, score, round(timer,3)]

data_model = dict()
data_confirm = dict()
data_preds = dict()
from sklearn.metrics import classification_report


model_path = 'ml_model'

# 1차 검색 테스트
for model in models:
    print('{} model 시작'.format(model))

    try:
        timer = time.time()
        model.fit(x_train,y_train)
        preds = model.predict(x_test)
        score = accuracy_score(y_test, preds)
        timer = time.time() - timer
        
        check_df.loc[len(check_df)] = [model.__class__.__name__, score, round(timer,3)]
        
        data_model[model.__class__.__name__] = model
        data_confirm[model.__class__.__name__] = classification_report(y_test, preds)
        data_preds[model.__class__.__name__] = preds
        with open(os.path.join(model_path,'{}.pickle'.format(model.__class__.__name__)),'wb') as f:
            pickle.dump(model, f)
    except:
        print('error model : {}'.format(model.__class__.__name__))


check_df

--- 
# 데이터 3중 분할 확장 테스트

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import pickle

# 1안 마지막 데이터만 선정
last_count_index = test_df.reset_index().sort_values('Sale Start').groupby('Game_ID')[['price','index']].last()['index'].unique()
running_df = pd.get_dummies(test_df)

train_running_df = running_df.drop(last_count_index)
test_running_df = running_df.iloc[last_count_index]

train_running_df = train_running_df.dropna()
test_running_df = test_running_df.dropna()
#running_df.iloc[last_count_index]   # 마지막 데이터
#running_df.drop(last_count_index)   # 그 외 데이터

x_train = train_running_df.drop(['Final price','Sale Start'],axis=1)
y_train = train_running_df['Final price']

x_test = test_running_df.drop(['Final price','Sale Start'],axis=1)
y_test = test_running_df['Final price']


# sklearn 에 적재되어 있는 알고리즘
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

# 설치가 필요한 앙상블 알고리즘 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# 평가함수
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# 분류 기본 알고리즘
#tree = DecisionTreeClassifier()     # 의사결정트리
logistic = LogisticRegression()     # 로지스틱회귀
#svc = SVC(probability=True)         # SVC
#knn = KNeighborsClassifier()        # KMean
#nb = GaussianNB()
#cat = CatBoostClassifier()

# Ensemble
#rfc = RandomForestClassifier(n_estimators=1000)         # 랜덤 포래스트
#bag = BaggingClassifier(n_estimators=100)               # bagging
#ada = AdaBoostClassifier(n_estimators=500)              # AdaBoost
#grad = GradientBoostingClassifier(n_estimators=500)     # GradientBoost
#xgb = XGBClassifier(n_estimators=500)                   # xgbBoost
#lgbm = LGBMClassifier()                                 # lightgbBoost


#tree.fit(x_train,y_train)
#models = [ tree,logistic,svc, knn, nb, rfc, bag, ada, grad, xgb, lgbm ]
#rfc.fit(x_train,y_train)
#nb.fit(x_train,y_train)
#preds = rfc.predict(x_test)
#classification_report(y_test, preds)
#xgb.fit(x_train, y_train)
#lgbm.fit(x_train, y_train)
#학습

logistic.fit(x_train,y_train)




c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_train.info()      # 18610
x_train.info()

x_train.select_dtypes(float)

In [18]:
test_df = pd.read_csv('테스트용파일.csv')
test_df

,Unnamed: 0,게임 ID,게임별 세일 순서,출시일수,세일 기간,세일 간격,최근 평가수,종합 평가수,총 판매금액,총 순이익,...,게임 장르_['Racing'],"게임 장르_['Simulation', 'Early Access']","게임 장르_['Simulation', 'Sports', 'Strategy']","게임 장르_['Simulation', 'Sports']","게임 장르_['Simulation', 'Strategy', 'Early Access']","게임 장르_['Simulation', 'Strategy']",게임 장르_['Simulation'],게임 장르_['Strategy'],게임 장르_['Utilities'],게임 장르_[]
0,0,10,1,8496,7,21,1267,146475,50000000.0,30000000.0,...,False,False,False,False,False,False,False,False,False,False
1,1,10,2,8496,13,32,1267,146475,50000000.0,30000000.0,...,False,False,False,False,False,False,False,False,False,False
2,2,10,3,8496,7,285,1267,146475,50000000.0,30000000.0,...,False,False,False,False,False,False,False,False,False,False
3,3,10,4,8496,8,21,1267,146475,50000000.0,30000000.0,...,False,False,False,False,False,False,False,False,False,False
4,4,10,5,8496,12,171,1267,146475,50000000.0,30000000.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26010,26010,985890,23,1376,7,98,82,14585,6000000.0,4000000.0,...,False,False,False,False,False,False,False,False,False,False
26011,26011,985890,24,1376,14,55,82,14585,6000000.0,4000000.0,...,False,False,False,False,False,False,False,False,False,False
26012,26012,985890,25,1376,11,64,82,14585,6000000.0,4000000.0,...,False,False,False,False,False,False,False,False,False,False
26013,26013,985890,26,1376,8,22,82,14585,6000000.0,4000000.0,...,False,False,False,False,False,False,False,False,False,False


In [45]:
game_id = 20

last_count_index = test_df.reset_index().sort_values('게임별 세일 순서').groupby('게임 ID')[['할인 가격(Y)','index']].last()['index'].unique()
find_price_df = test_df.iloc[last_count_index]

# 예측값
x_predict_test = find_price_df[find_price_df['게임 ID'] == game_id]

# 검색할 게임 가격 번호
find_price_df[find_price_df['게임 ID'] == game_id][-1:].index

x_find_test = find_price_df.drop(['할인 가격(Y)'],axis=1)
y_find_test = find_price_df['할인 가격(Y)']

# 알고리즘 시작
preds_price = logistic.predict(x_predict_test)

# 가격확인
confirm_price = y_find_test[x_find_test[x_find_test['게임 ID'] == game_id].index].values[0]

find_price_df['할인 가격(Y)']



ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 가격(원가)
- 게임 ID
- 게임 장르_['Action', 'Adventure', 'Casual', 'Indie', 'RPG', 'Early Access']
- 게임 장르_['Action', 'Adventure', 'Casual', 'Indie', 'RPG', 'Simulation', 'Strategy']
- 게임 장르_['Action', 'Adventure', 'Casual', 'Indie', 'RPG', 'Strategy']
- ...
Feature names seen at fit time, yet now missing:
- End Year_2012
- End Year_2013
- End Year_2014
- End Year_2015
- End Year_2016
- ...


In [44]:
find_price_df[find_price_df['게임 ID'] == id]

,Unnamed: 0,게임 ID,게임별 세일 순서,출시일수,세일 기간,세일 간격,최근 평가수,종합 평가수,총 판매금액,총 순이익,...,게임 장르_['Racing'],"게임 장르_['Simulation', 'Early Access']","게임 장르_['Simulation', 'Sports', 'Strategy']","게임 장르_['Simulation', 'Sports']","게임 장르_['Simulation', 'Strategy', 'Early Access']","게임 장르_['Simulation', 'Strategy']",게임 장르_['Simulation'],게임 장르_['Strategy'],게임 장르_['Utilities'],게임 장르_[]
6053,6053,20920,78,4647,15,40,0,61409,38000000.0,23000000.0,...,False,False,False,False,False,False,False,False,False,False


Index([5409], dtype='int64')

In [43]:

confirm_price = y_find_test[x_find_test[x_find_test['게임 ID'] == 20].index].values[0]
print(confirm_price)

find_price_df['할인 가격(Y)']

780


31        1050
5409       780
12892      780
15377      780
17358      780
         ...  
8439     14000
8440     17500
8442       880
8591      2720
8729     12500
Name: 할인 가격(Y), Length: 790, dtype: int64

In [30]:
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import pickle

# 1안 마지막 데이터만 선정
last_count_index = test_df.reset_index().sort_values('Sale Start').groupby('Game_ID')[['price','index']].last()['index'].unique()
running_df = pd.get_dummies(test_df)

train_running_df = running_df.drop(last_count_index)
test_running_df = running_df.iloc[last_count_index]

train_running_df = train_running_df.dropna()
test_running_df = test_running_df.dropna()
#running_df.iloc[last_count_index]   # 마지막 데이터
#running_df.drop(last_count_index)   # 그 외 데이터

x_train = train_running_df.drop(['Final price','Sale Start'],axis=1)
y_train = train_running_df['Final price']

x_test = test_running_df.drop(['Final price','Sale Start'],axis=1)
y_test = test_running_df['Final price']


last_count_index = test_df.reset_index().sort_values('게임별 세일 순서').groupby('게임 ID')[['할인 가격(Y)','index']].last()['index'].unique()
find_price_df = test_df.iloc[last_count_index]

x_find_test = find_price_df.drop(['할인 가격(Y)'],axis=1)
y_find_test = find_price_df['할인 가격(Y)']

# 탐색할 게임 id
id = 20920

# 검색할 게임 
preds_price = logistic.predict(x_find_test[x_find_test['게임 ID'] == id])

# 검색할 게임 가격 번호
test_df[test_df['게임 ID'] == id][-1:].index

# 검색한 가격 확인
confirm_price = y_find_test[x_find_test[x_find_test['게임 ID'] == id].index].values[0]

if confirm_price == preds_price:
    print("가격을 맞췄습니다.", confirm_price)

print('확인가격 : ', confirm_price)
print('예측가격 : ', preds_price[0])

from sklearn.metrics import classification_report

classification_report(preds_price,y_test[x_test[x_test['게임 ID'] == id].index])




ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 가격(원가)
- 게임 ID
- 게임 장르_['Action', 'Adventure', 'Casual', 'Indie', 'RPG', 'Early Access']
- 게임 장르_['Action', 'Adventure', 'Casual', 'Indie', 'RPG', 'Simulation', 'Strategy']
- 게임 장르_['Action', 'Adventure', 'Casual', 'Indie', 'RPG', 'Strategy']
- ...
Feature names seen at fit time, yet now missing:
- End Year_2012
- End Year_2013
- End Year_2014
- End Year_2015
- End Year_2016
- ...


In [15]:

# 1안 마지막 데이터만 선정
last_count_index = test_df.reset_index().sort_values('Sale Start').groupby('Game_ID')[['price','index']].last()['index'].unique()
test_running_df = running_df.iloc[last_count_index]
test_running_df = test_running_df.dropna()

x_test = test_running_df.drop(['Final price','Sale Start'],axis=1)
y_test = test_running_df['Final price']

# 탐색할 게임 id
id = 20920

# 검색할 게임 
preds_price = logistic.predict(x_test[x_test['Game_ID'] == id])

# 검색할 게임 가격 번호
temp_df[temp_df['Game_ID'] == id][-1:].index

# 검색한 가격 확인
confirm_price = y_test[x_test[x_test['Game_ID'] == id].index].values[0]

if confirm_price == preds_price:
    print("가격을 맞췄습니다.", confirm_price)

print('확인가격 : ', confirm_price)
print('예측가격 : ', preds_price[0])

from sklearn.metrics import classification_report

classification_report(preds_price,y_test[x_test[x_test['Game_ID'] == id].index])



확인가격 :  3150
예측가격 :  5250


c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\GJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

'              precision    recall  f1-score   support\n\n        3150       0.00      0.00      0.00       0.0\n        5250       0.00      0.00      0.00       1.0\n\n    accuracy                           0.00       1.0\n   macro avg       0.00      0.00      0.00       1.0\nweighted avg       0.00      0.00      0.00       1.0\n'

In [ ]:
#y_train.loc(temp_df[temp_df['Game_ID'] == 995460][-1:].index)

#y_train[temp_df[temp_df['Game_ID'] == 995460][-1:].index[0]]
#temp_df[temp_df['Game_ID'] == 995460][-1:].index[0]
temp_df[temp_df['Game_ID'] == 995460][-1:].index

---

In [ ]:
x_train